In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

#Cargar datos frescos (Sin get_dummies, sin trucos raros)
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

El problema de XGBoost: Al usar get_dummies (One-Hot Encoding), convertiste 1 columna "Barrio" en 25 columnas llenas de ceros (Barrio_A, Barrio_B...). Esto crea una matriz "dispersa" (muchos huecos vacíos), y a los árboles les cuesta encontrar patrones fuertes ahí.

La magia de CatBoost: No usa get_dummies. Usa una técnica llamada "Ordered Target Statistics". Básicamente, calcula el precio promedio de cada barrio y usa ese número para entrenar. Mantiene la información compacta y densa.

In [3]:
# Limpieza Básica (Rellenar huecos)
# Para columnas de TEXTO: Rellenamos con "None" (así lo trata como una categoría)
# Para columnas NUMÉRICAS: Rellenamos con 0
for col in df_train.columns:
    if col == 'SalePrice':
        df_train[col] = df_train[col].fillna(0)
    else:    
        if df_train[col].dtype == 'object':
            df_train[col] = df_train[col].fillna("None")
            df_test[col] = df_test[col].fillna("None")
        else:
            df_train[col] = df_train[col].fillna(0)
            df_test[col] = df_test[col].fillna(0)

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1459 non-null   object 
 3   LotFrontage    1459 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          1459 non-null   object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1459 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [ ]:
#Preparamos X e y (Usando Logaritmo en el precio como aprendimos)
X = df_train.drop(['SalePrice', 'Id'], axis=1)
y_log = np.log1p(df_train['SalePrice'])
X_test_kaggle = df_test.drop(['Id'], axis=1)

# Identificar cuáles son las columnas de texto (Cat Features)
# CatBoost necesita saber el índice o nombre de estas columnas
indices_texto = np.where(X.dtypes == 'object')[0]

#ENTRENAMIENTO
print("Entrenando CatBoost... (Verás cómo baja el error)")
modelo_cb = CatBoostRegressor(
    iterations=1000,     # Número de árboles correctores
    learning_rate=0.05,  # Velocidad de aprendizaje (más bajo = más preciso pero lento)
    depth=6,             # Profundidad de los árboles
    loss_function='RMSE',
    verbose=100          # Imprime cada 100 iteraciones
)

#Le pasamos cat_features
modelo_cb.fit(X, y_log, cat_features=indices_texto)

#Predicción y Guardado
pred_log = modelo_cb.predict(X_test_kaggle)
pred_final = np.expm1(pred_log) # Deshacemos el logaritmo

sub = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': pred_final})
sub.to_csv('house_catboost.csv', index=False)
print("¡Archivo 'house_catboost.csv' listo!")

Entrenando CatBoost... (Verás cómo baja el error)
0:	learn: 0.3864116	total: 199ms	remaining: 3m 18s
100:	learn: 0.1155127	total: 5.54s	remaining: 49.3s
200:	learn: 0.0979681	total: 10.9s	remaining: 43.4s
300:	learn: 0.0876329	total: 16.3s	remaining: 37.9s
400:	learn: 0.0793504	total: 21.5s	remaining: 32.1s
500:	learn: 0.0729715	total: 26.7s	remaining: 26.6s
600:	learn: 0.0668143	total: 31.8s	remaining: 21.1s
700:	learn: 0.0616129	total: 37s	remaining: 15.8s
800:	learn: 0.0573103	total: 42.4s	remaining: 10.5s
900:	learn: 0.0536828	total: 47.8s	remaining: 5.25s
999:	learn: 0.0503432	total: 52.8s	remaining: 0us
¡Archivo 'house_catboost.csv' listo!
